In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.tools import tool
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [1]:
import os

from dotenv import load_dotenv

# .env 파일 로드, 환경 변수에서 API 키 읽기
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [2]:
from langchain_openai import ChatOpenAI

# LLM 초기화
llm = ChatOpenAI(
    model_name="(입력 필요) MODEL",  # e.g. "openai/gpt-4o-mini", "openai/gpt-5"
    base_url="(입력 필요) BASE_URL",  # e.g. "https://mlapi.run.../v1"
    temperature=0,
)

# 1. LangGraph의 구조와 그래프 시각화
- Agent State, Tools, Nodes, Edges, Compile 등을 배웁니다.  
- Agent의 workflow를 시각화하여 표현합니다.

## LangGraph로 간단한 챗봇 구현
LangChain으로 구현된 간단한 챗봇을 LangGraph로 구현해봅니다.  
챗봇을 구현하면서 다음 내용들을 학습합니다.  
- `StateGraph` : Agent에서 사용할 데이터 정의
- `Node` : 실제 특정 작업 수행
- `ToolNode` : Agent가 사용할 수 있는 외부 tool 정의
- `Edge` : Node들을 연겨하고, Agent 작동 플로우 정의

### Define Agent State
Graph 내에서 유통될 state(데이터)를 정의합니다.  
- AgentState Class 정의
  - `class AgentState(TypeDict)`
    -  `AgentState`라는 이름의 딕셔너리 타입을 정의
    - 이 딕셔너리에 Agent 상태가 담김
  - `messages: Annotated[str, add_messages]`
    - `messages`
      - AgentState 딕셔너리는 `messages`라는 키를 갖는다는 의미
    - `Annotated[str, add_messages]`
      - `messages` 키에 값이 업데이트 되는 방식 지정
      - 새로운 메시지가 들어왔을 때 뒤에 추가하는 방식 사용 (`add_messages`)

In [ ]:
from typing import Annotated, TypedDict

from langgraph.graph.message import add_messages

In [ ]:
class AgentState(TypedDict):
    # The add_messages function defines how an update should be processed
    # Default is to replace. add_messages says "append"
    messages: Annotated[str, add_messages]

### Define Tools
- 문서를 불러와 Chunking하고 임베딩하여 Vector Store에 저장합니다.  
- `retriever = vectorstore.as_retriever()`
  - 생성된 vectorstore를 기반으로 검색기(Retriever)를 만듭니다.  
  - 이렇게 만들어진 검색기는 쿼리와 가장 관련 있는 텍스트 조각을 DB에서 신속하게 찾아주는 역할을 수행합니다.  

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# HuggingFace에서 제공하는 임베딩 모델을 로드합니다.
# 'all-MiniLM-L6-v2'는 작고 빠르면서도 성능이 좋은 범용 모델입니다.
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

loader = PyPDFLoader("RAG_paper.pdf")
pages = loader.load_and_split()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(pages)

vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_model)
retriever = vectorstore.as_retriever()

### Define Nodes
- 현재 State(현재 대화 상태)를 입력받아 RAG 파이프라인을 통해 답변을 생성하고, 그 결과를 다시 State(대화 상태)에 추가하여 반환하는 노드 정의

In [ ]:
def generate_answer(state):

    print("---GENERATE ANSWER FROM RAG---")

    messages = state["messages"]
    query = messages[-1].content  # user query

    system_prompt = """당신은 질문-답변을 담당하는 전문가 입니다. 다음 정보를 활용하여 질문에 답을 하시오.
        모르면 모른다고 답하고, 답변은 간결하게 하시오.
        {context}"""

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            ("human", "{input}"),
        ]
    )

    question_answer_chain = create_stuff_documents_chain(llm, prompt)
    rag_chain = create_retrieval_chain(retriever, question_answer_chain)

    print("Input query for RAG: ", query)

    response = rag_chain.invoke({"input": query})
    print(response)

    return {"messages": [response["answer"]]}

### Workflow 구현 및 시각화
- 뼈대가 되는 `StateGraph`를 먼저 정의합니다.  
- 이후 invoke, batch, stream 함수를 실행하여 데이터를 State의 형태로 주입합니다.  

In [ ]:
from langgraph.graph import END, START, StateGraph
from langgraph.prebuilt import ToolNode

# StateGraph 객채를 정의합니다.
workflow = StateGraph(AgentState)

# 노드 생성
# : 생성된 StateGraph 객체에 'rag'라는 이름으로 node를 생성합니다.
workflow.add_node("rag", generate_answer)  # 답변 생성

In [ ]:
# 워크플로우가 시작(START)되면 가장 먼저 'rag' 노드로 이동하도록 설정합니다.
workflow.add_edge(START, "rag")

# 노드의 작업이 끝나면 워크플로우를 종료(END)하도록 설정합니다.
workflow.add_edge("rag", END)

# compile하여 그래프의 아키텍쳐를 고정합니다.
graph = workflow.compile()

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph(xray=True).draw_mermaid_png()))
except Exception:
    # This requires some extra dependencies and is optional
    pass

In [ ]:
inputs = {"messages": ["what is the summary of the paper??"]}

In [ ]:
output = graph.invoke(inputs)

In [ ]:
print(output["messages"][-1].content)

# 2. ToolNode
- 원하는 도구를 사용할 수 있게 해주는 Tool node의 활용 방법을 익힙니다.  
- RAG를 tool의 형태로 변환하여 상황에 따라 동적으로 활용할 수 있도록 합니다.

### ToolNode
- `@tool`
  - 정의된 함수를 LLM이 사용할 수 있는 tool로 만들어줍니다.
  - input을 통해 정해진 tool을 실행하며, output을 제공해줍니다.

In [ ]:
# tool이라는 magic function을 활용하여 사용할 툴을 인지


@tool
def get_weather(location: str):
    """Call to get the current weather."""
    if location.lower() in ["sf", "san francisco"]:
        return "It's 20 degrees and foggy."
    else:
        return "It's 30 degrees and sunny."


@tool
def get_coolest_cities():
    """Get a list of coolest cities"""
    return "nyc, sf"


# 여러 개의 tool들을 리스트 형태로 제공
tools = [get_weather, get_coolest_cities]
# 다수의 tool을 갖고 있는 node가 생성되며, 인풋에 따라 사용할 tool 결정
tool_node = ToolNode(tools)

`bind_tools` 메서드를 호출해 LLM에 도구들을 인식시키고 tool을 들고 있는 llm_with_tools를 정의합니다.   

In [ ]:
llm_with_tools = llm.bind_tools(tools)

LLM이 쿼리를 보고 get_weather tool을 호출해야 한다고 판단하고 tool_cals 인자가 채워져서 반환됩니다.

In [ ]:
from pprint import pprint

output = llm_with_tools.invoke("what's the weather in sg?")
pprint(output)

만약 LLM을 사용하지 않고 다른 preprocessing step을 추가하여 tool을 직접 실행하고자 한다면, 다음과 같이 실행할 수도 있습니다.  

In [ ]:
# from langchain_core.messages import AIMessage
#
# message_with_single_tool_call = AIMessage(
#     content="",
#     tool_calls=[
#         {
#             "name": "get_weather",
#             "args": {"location": "sf"},
#             "id": "random_ID",
#             "type": "tool_call",
#         }
#     ],
# )

# tool_node.invoke({"messages": [message_with_single_tool_call]})

## Retrieval Tool 구축

In [ ]:
@tool
def build_and_retrieve(query: str):
    """A knowledge base for retrieving information related to muscle gains"""
    loader = PyPDFLoader("RAG_paper.pdf")
    pages = loader.load_and_split()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(pages)

    vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_model)
    retriever = vectorstore.as_retriever()

    return retriever.invoke(query)


@tool
def get_weather(location: str):
    """Call to get the current weather."""
    if location.lower() in ["sf", "san francisco"]:
        return "It's 20 degrees and foggy."
    else:
        return "It's 30 degrees and sunny."


@tool
def get_coolest_cities():
    """Get a list of coolest cities"""
    return "nyc, sf"


tools = [build_and_retrieve, get_weather, get_coolest_cities]
tool_node = ToolNode(tools)

`message_with_single_tool_call`에 tool을 직접 지정해서 Tool node에 전달할 수 있습니다.

In [ ]:
from langchain_core.messages import AIMessage

message_with_single_tool_call = AIMessage(
    content="",
    tool_calls=[
        {
            "name": "build_and_retrieve",
            "args": {"query": "how can I gain more muscle"},
            "id": "random_ID",
            "type": "tool_call",
        }
    ],
)

output = tool_node.invoke({"messages": [message_with_single_tool_call]})
output

In [ ]:
output["messages"][0].content

In [ ]:
from chromadb import chromadb

chromadb.api.client.SharedSystemClient.clear_system_cache()

LLM with tools를 이용하면 다음과 같습니다.

In [ ]:
llm_with_tools = llm.bind_tools(tools)

In [ ]:
# tool과 상관 없는 input이 들어올 경우, 일반 chat 모델과 동일한 결과 생성
output = llm_with_tools.invoke("What is the best way to learn science?")
output

In [ ]:
# tool과 상관 없는 Input이기 때문에 tool calling을 실행하지 않음
# tool calling은 정의한 tool의 description을 바탕으로 LLM이 선택
output.tool_calls

In [ ]:
tool_input = llm_with_tools.invoke("How can I gain more muscle?")
tool_input.tool_calls

In [ ]:
tool_output = tool_node.invoke({"messages": [tool_input]})
tool_output

In [ ]:
pprint(tool_output["messages"][0].content)